In [314]:
from mtspHeuristic import *
from mtspConstruction import *
import random
import numpy as np

In [315]:
def singleVertexRemoval(route, vertex, obj_function,distance_matrix):
    # Premissa: vertex está em route
    # Adição da origem da rota ao vetor de rota
    route = [1] + route + [1]

    # Etapa de busca sequencial
    vertex_index = -1
    for item in range(1,len(route)):
        if route[item] == vertex:
            vertex_index = item

    city_before = route[vertex_index-1]
    city_removed = route[vertex_index]
    city_after = route[vertex_index+1]
    #print(city_before, city_removed, city_after)
    #city = 2
    obj_function = obj_function - distance_matrix[city_before-1][city_removed-1] - distance_matrix[city_removed-1][city_after-1] + distance_matrix[city_before-1][city_after-1]
    # Truncamento da rota, para retirar os índices 1 do começo e do fim
    route = route[:vertex_index] + route[vertex_index+1:]
    route = route[1:-1]
    return route, obj_function

In [316]:
def removalOfVertex(solutions,vertex_lists,obj_function,distance_matrix):
    for vertex in vertex_lists:
        for item in range(len(solutions)):
            #print("vertex", vertex, "in route", item, "?", vertex in solutions[item])
            if vertex in solutions[item]:
                solutions[item], obj_function[item] = singleVertexRemoval(solutions[item],vertex,obj_function[item],distance_matrix)
    return solutions, obj_function

In [317]:
test_graph = [[0,8,5,10,6,3,13,2,10,6,3,4,9,6,17],
         [8,0,9,6,12,7,4,9,15,9,10,6,5,9,11],
         [5,9,0,10,9,9,7,18,7,4,7,3,11,4,10],
         [10,6,10,0,18,2,10,6,5,4,9,12,8,8,10],
         [6,12,9,18,0,9,10,11,11,12,6,5,6,8,9],
         [3,7,9,2,9,0,6,8,9,10,5,2,7,10,6],
         [13,4,7,10,10,6,0,12,8,3,7,4,10,6,9],
         [2,9,18,6,11,8,12,0,7,3,6,18,17,9,2],
         [10,15,7,5,11,9,8,7,0,10,8,3,14,10,3],
         [6,9,4,4,12,10,3,3,10,0,12,5,9,6,3],
         [3,10,7,9,6,5,7,6,8,12,0,19,8,8,11],
         [4,6,3,12,5,2,4,18,3,5,19,0,12,2,9],
         [9,5,11,8,6,7,10,17,14,9,8,12,0,5,6],
         [6,9,4,8,8,10,6,9,10,6,8,2,5,0,2],
         [17,11,10,10,9,6,9,2,3,3,11,9,6,2,0]]

solutions = [[3, 6, 13, 12, 14, 9, 11, 15, 5],
[8, 4, 7],
[2, 10]]

obj_function = [79, 31, 23]

a,b = removalOfVertex(solutions,[13,9,4,2],obj_function,test_graph)
a, b, objective_function_vector_bf(a, test_graph)

([[3, 6, 12, 14, 11, 15, 5], [8, 7], [10]], [52, 27, 12], [52, 27, 12])

In [319]:
def destruction_reconstruction_LNS(solutions,obj_function,distance_matrix,alpha = 0.2,removal_amount = 3):
    
    solutions_copy = solutions.copy()
    obj_function_copy = obj_function.copy()
    
    cities_amount = len(distance_matrix)
    cities_to_be_removed = list(range(2,cities_amount+1))

    # Etapa de escolha de vértices que não podem ser removidos
    # (Evita rotas vazias) 
    for route in solutions_copy:
        cities_to_be_removed.remove(random.choice(route))         

    removed_cities = random.sample(cities_to_be_removed, removal_amount)

    # Bugs encontrados pela maneira determinística (exclusão de "rotas pequenas"):
    # Pode haver uma instância em que todas as cidades removidas estejam em uma mesma rota   

    solutions_copy, obj_function_copy = removalOfVertex(solutions_copy,removed_cities,obj_function_copy,distance_matrix)

    solutions_copy = [[1] + i + [1] for i in solutions_copy]

    for city_candidate in removed_cities: # Para cada cidade removida
        insertion_candidates = []
        for route_index in range(len(solutions)): # Para cada rota
            route = solutions_copy[route_index]
            for city_index in range(1, len(route)): # Para cada cidade naquela rota
                city_in_route = route[city_index]
                city_before_in_route = route[city_index-1]
                insertion_cost = distance_matrix[city_in_route-1][city_candidate-1] + distance_matrix[city_candidate-1][city_before_in_route-1] - distance_matrix[city_in_route-1][city_before_in_route-1]
                insertion_candidates.append([insertion_cost, city_index, city_candidate, route_index])

        insertion_candidates.sort(key=lambda x: x[0])
        #print(insertion_candidates)
        min_sol_value = insertion_candidates[0][0]
        max_sol_value = insertion_candidates[-1][0]
        gap_in_solutions = max_sol_value - min_sol_value

        insertion_candidates_filtered = []

        for item in insertion_candidates:
            if item[0] <= min_sol_value + gap_in_solutions*alpha:
                #print(item, " ", min_sol_value + gap_in_solutions*alpha)
                insertion_candidates_filtered.append(item)
        #print("Filtered Candidates: ", insertion_candidates_filtered)

        insertion_chosen_solution = random.choice(insertion_candidates_filtered)
        obj_function_difference = insertion_chosen_solution[0]
        city_index_to_insert = insertion_chosen_solution[1]
        city_to_insert = insertion_chosen_solution[2]
        route_to_increment_index = insertion_chosen_solution[3]

        solutions_copy[route_to_increment_index] = insert_in_vector(solutions_copy[route_to_increment_index], city_to_insert, city_index_to_insert)
        obj_function_copy[route_to_increment_index] += obj_function_difference
    solutions_copy = [item[1:-1] for item in solutions_copy]
    #print("Solutions: ")
    #print_routes(solutions_copy)
    return solutions_copy, obj_function_copy

## Macro código de remoção por Worst Removal e Cluster Removal

In [ ]:
#def WorstRemoval(solution, numToRemove):
#    p = 4
#    A = []
#    ListRemovedCities = []
#    # Para cada uma das rotas da EFO:
#    for k in range(numToRemove):
#        #Cria lista A ordenada por custo de remocao
#        sorteio = random.rand()
#        indice = int(len(A) * sorteio^(p))
#        citieToRemove = A[indice]
#        remove(solution, citieToRemove)
#        A.remove(citieToRemove)
#        ListRemovedCities.append(citieToRemove)
#
#    return ListRemovedCities
#
#def ClusterRemoval(solution, numToRemove):
#    p = 4
#    A = []
#    citieRef = sample(listaDeCidades) # Sorteie uma cidade da sua lista
#    ListRemovedCities = []
#    #Crie aqui uma lista A orenada de acordo com o custo de distância das demais cidades à cidade cluster
#    # (incluindo a cidade cluster)
#    # Para cada uma das rotas da EFO:
#    for k in range(numToRemove):
#        sorteio = random.rand()
#        indice = int(len(A) * sorteio^(p))
#        citieToRemove = A[indice]
#        remove(solution, citieToRemove)
#        A.remove(citieToRemove)
#        ListRemovedCities.append(citieToRemove)
#
#    return ListRemovedCities

In [320]:
solutions = [[3, 6, 13, 12, 14, 9, 11, 15, 5],
[8, 4, 7],
[2, 10]]

obj_function = [79, 31, 23]

In [321]:
distance_matrix = test_graph
a, b = destruction_reconstruction_LNS(solutions,obj_function,distance_matrix)
a, b, objective_function_vector_bf(a, distance_matrix), [i1 == i2 for (i1, i2) in zip(b, objective_function_vector_bf(a, distance_matrix))]

([[3, 13, 12, 14, 9, 11, 15, 8], [6, 4, 7, 5], [2, 10]],
 [63, 31, 23],
 [63, 31, 23],
 [True, True, True])

# Etapa de Simulated Annealing

In [322]:
T_initial = 50000000
cooling_rate = 0.9
max_iter = 1000
T_minimum = 0.001
max_iter_no_improve = 6000

number_of_salesmen = 3
solution, obj_function = greedy_insertion_construction(distance_matrix, number_of_salesmen)
print_routes(solution)
initial_OF = objective_function_minsum(obj_function)

[3, 10, 7, 2, 12, 9, 4, 6, 8, 15, 13, 5]
[11]
[14]


In [308]:
iter_number = 0
Temperature = T_initial
best_OF = initial_OF
current_OF = initial_OF
solution_current = solution.copy()
obj_function_current = obj_function.copy()
solution_best = solution.copy()
obj_function_best = obj_function.copy()
while(Temperature > T_minimum):
    iter_number = 0
    while(iter_number < max_iter):
        # Parte específica
        solution_rebuilt, obj_function_rebuilt = destruction_reconstruction_LNS(solution_current,obj_function_current,distance_matrix)
        neighbor_OF = objective_function_minsum(obj_function_rebuilt)
        if neighbor_OF < current_OF:
            current_OF = neighbor_OF
            solution_current = solution_rebuilt.copy()
            obj_function_current = obj_function_rebuilt.copy()
            if neighbor_OF < best_OF:
                best_OF = neighbor_OF
                solution_best = solution_rebuilt.copy()
                obj_function_best = obj_function_rebuilt.copy()
                print("Found best solution at T = %.3f with value %.3f" % (Temperature, best_OF) )
        #FIM
        else:
            delta = neighbor_OF - current_OF
            if random.random() < np.exp(-delta/Temperature):
                # Parte específica
                current_OF = neighbor_OF
                solution_current = solution_rebuilt.copy()
                obj_function_current = obj_function_rebuilt.copy()
                # FIM
        iter_number += 1
    #print("Iter done")
    Temperature *= cooling_rate
                

print("Best OF: %d \nCurrent OF after halt: %d" % (best_OF, current_OF))
print("Best Routes: ")
print_routes(solution_best)
print("Current Routes: ")
print_routes(solution_current)

Found best solution at T = 50000000.000 with value 71.000
Found best solution at T = 50000000.000 with value 69.000
Found best solution at T = 50000000.000 with value 68.000
Found best solution at T = 50000000.000 with value 66.000
Found best solution at T = 50000000.000 with value 64.000
Found best solution at T = 50000000.000 with value 63.000
Found best solution at T = 50000000.000 with value 61.000
Found best solution at T = 49950000.000 with value 60.000
Found best solution at T = 49950000.000 with value 59.000
Found best solution at T = 49900050.000 with value 58.000
Best OF: 58 
Current OF after halt: 58
Best Routes: 
[8]
[11]
[6, 4, 9, 15, 14, 12, 3, 10, 7, 2, 13, 5]
Current Routes: 
[8]
[11]
[5, 13, 2, 7, 10, 3, 12, 14, 15, 9, 4, 6]


In [309]:
print("Best OF: %d \nCurrent OF after halt: %d" % (best_OF, current_OF))
print("Best Routes: ")
print_routes(solution_best)

print("Current Routes: ")
print_routes(solution_current)
print("With objective function of",current_OF)

Best OF: 58 
Current OF after halt: 58
Best Routes: 
[8]
[11]
[6, 4, 9, 15, 14, 12, 3, 10, 7, 2, 13, 5]
Current Routes: 
[8]
[11]
[5, 13, 2, 7, 10, 3, 12, 14, 15, 9, 4, 6]
With objective function of 58


### All time best (3 salesmen):

Metaheuristic: LNS (Large Neighborhood Search)<br>
Best OF: 58 <br>
Current OF after halt: 58 <br>
Best Routes: <br>
[8]<br>
[11]<br>
[6, 4, 9, 15, 14, 12, 3, 10, 7, 2, 13, 5]<br> <br>
Current Routes: <br>
[8] <br>
[11] <br>
[5, 13, 2, 7, 10, 3, 12, 14, 15, 9, 4, 6] <br>
With objective function of 58
#### ---------------------------------------------------------------
Metaheuristic: BRKGA (_Biased Random Key Genetic Algorithm_)<br> 
Best routes: <br> 
[11]<br> 
[6, 4, 9, 15, 14, 12, 3, 10, 7, 2, 13, 5]<br> 
[8]<br> 
With objective function of 58<br> 


In [1]:
# random removal
L_minus = []
beta = round(random.random()*0.3+0.1,2)
beta

NameError: name 'random' is not defined

In [323]:
objective_function_vector_bf(solution_current,test_graph)

[4, 6, 48]